In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [3]:
!ls

homework.md   model.bin     starter.ipynb


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
print(y_pred)

[18.52778307 23.06578208 33.68635854 ... 11.89045938 15.10268128
  9.46059157]


In [13]:
y_pred.std()

5.28140357655334

In [14]:
year = 2022
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

output_file = "output.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [17]:
import os

file_name = "output.parquet"
file_stats = os.stat(file_name)

print(f"File size is {file_stats.st_size} bytes, "
      f"{file_stats.st_size / 1024:.1f} kB, "
      f"{file_stats.st_size / (1024 * 1024):.3f} MB.")

File size is 59994935 bytes, 58588.8 kB, 57.216 MB.
